<a href="https://colab.research.google.com/github/euguroglu/Machine-Learning-Projects/blob/master/Pytorch_Mobile_phone_price_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sys
from sklearn.datasets import load_boston
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import math
import torch.nn.functional as F
from sklearn.metrics import precision_score

## Data Preperation

In [ ]:
df = pd.read_csv('../input/mobile-price-classification/train.csv')

In [ ]:
df = df.sort_values(by=['price_range'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['Res'] = df['px_height']*df['px_width']

In [ ]:
X = df.drop(['px_height','px_width'],1)

In [ ]:
y = df['price_range']

In [ ]:
X.head()

In [ ]:
y.value_counts()

## Correlation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
corr_matrix = X.corr()
corr_matrix
plt.figure(figsize=(12,8))
sns.heatmap(corr_matrix, annot=True, cmap='YlGnBu')
plt.show()

In [ ]:

x = df[['battery_power','ram']]

Y = y.values
plt.scatter(x.values[:,0],x.values[:,1],c=Y)
plt.xlabel('Ram')
plt.ylabel('Battery')
plt.title('Price Range Distrubtion')

In [ ]:
X = X[['ram','battery_power','Res']]

In [ ]:
X = X.to_numpy()

In [ ]:
y = y.to_numpy().reshape(-1,1)

In [ ]:
plt.scatter(X[:,0],X[:,1], c=y)
plt.xlabel('Ram')
plt.ylabel('Battery Power')
plt.title('Result Distribution')

In [ ]:
class Model(nn.Module):

  def __init__(self,in_features=3,h1=8,h2=9,out_features=4):
    # how many layers?
    super().__init__()
    
    self.fc1 = nn.Linear(in_features,h1)
    self.fc2 = nn.BatchNorm1d(h1)
    self.fc3 = nn.Linear(h1,h2)
    self.fc4 = nn.BatchNorm1d(h2)
    self.out = nn.Linear(h2,out_features)

   
    
  def forward(self,x):
    
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = self.out(x)
    return x

In [ ]:
model = Model()

In [ ]:
model

In [ ]:
X_train = torch.FloatTensor(X)

In [ ]:
y_train = torch.LongTensor(y).reshape(-1)

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
epochs = 2000
losses = []

for i in range(epochs):

  # forward and get a prediction
  y_pred = model.forward(X_train)
  # calculate loss/error
  loss = criterion(y_pred,y_train)
  losses.append(loss)
  if i%100==0:
    print('Epoch: {} and Loss: {}'.format(i,loss))
  #Backpropagation
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
y_item_2 = []
for i in range(2000):
  y_item_2.append(y_pred[i].argmax())

In [ ]:
y_item_2 = np.array(y_item_2)

## Checking accuracy

In [ ]:
correct = 0
for i in range(2000):
  if y_item_2[i]==y_train[i]:
    correct = correct + 1
print('Accuracy: {}'.format(correct*100/2000))

## Checking loss

In [ ]:
plt.plot(range(epochs),losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [ ]:
df_2 = pd.read_csv('../input/mobile-price-classification/test.csv')

In [ ]:
df_2['Res'] = df_2['px_height']*df_2['px_width']

In [ ]:
X_val = df_2[['ram','battery_power','Res']]

In [ ]:
X_val = X_val.to_numpy()

In [ ]:
X_test = torch.FloatTensor(X_val)

## Evaluation on test data

In [ ]:
# TO EVALUATE THE ENTIRE TEST SET
with torch.no_grad():
    y_pred_2 = model.forward(X_test)
    


In [ ]:
from sklearn.svm import SVC
from mlxtend.plotting import plot_decision_regions

In [ ]:
y_item = []
for i in range(1000):
  y_item.append(y_pred_2[i].argmax())

In [ ]:
y_item = np.array(y_item)

In [ ]:
X_test_plot = X_test[:,[0,1]]

## Test data Classification

In [ ]:
svm = SVC(C=0.5, kernel='linear')
svm.fit(X_test_plot, y_item)

In [ ]:
plot_decision_regions(X_test_plot.numpy(), y_item, clf=svm, legend=2)
plt.xlabel('RAM')
plt.ylabel('Battery Power')
plt.title('Decision Boundary')
plt.show()